<a href="https://colab.research.google.com/github/HardikDk121/Brain-Tumor-Classification/blob/main/CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
%pip install tensorflow matplotlib kagglehub

In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import os

In [25]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("briscdataset/brisc2025")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'brisc2025' dataset.
Path to dataset files: /kaggle/input/brisc2025


In [26]:
!find /kaggle/input/brisc2025 -maxdepth 3

/kaggle/input/brisc2025
/kaggle/input/brisc2025/brisc2025
/kaggle/input/brisc2025/brisc2025/manifest.json
/kaggle/input/brisc2025/brisc2025/README.md
/kaggle/input/brisc2025/brisc2025/segmentation_task
/kaggle/input/brisc2025/brisc2025/segmentation_task/test
/kaggle/input/brisc2025/brisc2025/segmentation_task/train
/kaggle/input/brisc2025/brisc2025/classification_task
/kaggle/input/brisc2025/brisc2025/classification_task/test
/kaggle/input/brisc2025/brisc2025/classification_task/train
/kaggle/input/brisc2025/brisc2025/manifest.csv
/kaggle/input/brisc2025/brisc2025/manifest.json.sha256
/kaggle/input/brisc2025/brisc2025/manifest.csv.sha256


In [27]:

# ==========================================
# 1. CONFIGURATION & SETUP
# ==========================================
# Define dataset paths and image parameters
# Structure should be:
#   Train/ -> [tumor, notumor]
#   Test/  -> [tumor, notumor]
TRAIN_DIR = '/kaggle/input/brisc2025/brisc2025/classification_task/train'
TEST_DIR = '/kaggle/input/brisc2025/brisc2025/classification_task/test'
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 32
EPOCHS = 15
LEARNING_RATE = 0.0001 # Slower learning rate for better stability

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

After mounting, you can save your model to a specific folder in your Google Drive, for example, to a folder named `my_models`:

In [ ]:
model.save('/content/drive/MyDrive/my_models/binary_brain_tumor_detector.h5')
print("\n💾 Model saved to Google Drive: '/content/drive/MyDrive/my_models/binary_brain_tumor_detector.h5'")

### 2. Data Preprocessing & Augmentation

In [28]:
# Training Data: Apply augmentation to artificially increase dataset size
train_datagen = ImageDataGenerator(
    rescale=1./255,             # Normalize pixel values (0-1)
    rotation_range=10,          # Slight rotation
    width_shift_range=0.1,      # Shift image horizontally
    height_shift_range=0.1,     # Shift image vertically
    shear_range=0.1,            # Shear transformation
    zoom_range=0.1,             # Slight zoom
    horizontal_flip=True,       # Flip horizontally
    fill_mode='nearest'
)

# Testing Data: ONLY rescale. Never augment validation data.
test_datagen = ImageDataGenerator(rescale=1./255)

# Load Images
print("Loading Training Data...")
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',        # ⚠️ CRITICAL: Use 'binary' for 2 classes
    shuffle=True
)

print("\nLoading Testing Data...")
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',        # ⚠️ CRITICAL: Use 'binary' for 2 classes
    shuffle=False
)

Loading Training Data...
Found 5000 images belonging to 4 classes.

Loading Testing Data...
Found 1000 images belonging to 4 classes.


### 3. Build Binary CNN Architecture

In [29]:
def build_binary_cnn():
    model = Sequential([
        Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3)),

        # Block 1: Feature Extraction
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),

        # Block 2: Deeper Features
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),

        # Block 3: Complex Patterns
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),

        # Flattening
        Flatten(),

        # Fully Connected Layers
        Dense(128, activation='relu'),
        Dropout(0.5), # Regularization to prevent overfitting

        # Output Layer
        # ⚠️ CRITICAL: 1 neuron with 'sigmoid' for Binary Classification
        Dense(1, activation='sigmoid')
    ])

    return model

model = build_binary_cnn()

### 4. Compile Model

In [30]:
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy', # ⚠️ CRITICAL: Loss function for binary tasks
    metrics=['accuracy']
)

# Show architecture
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    12,845,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,938,561 (49.36 MB)

 Trainable params: 12,938,561 (49.36 MB)

 Non-trainable params: 0 (0.00 B)

### 5. Train Model

In [31]:
print("\n🚀 Starting Training...")
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=test_generator,
    validation_steps=test_generator.samples // BATCH_SIZE,
    verbose=1
)


🚀 Starting Training...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
118/156 ━━━━━━━━━━━━━━━━━━━━ 3:11 5s/step - accuracy: 0.2607 - loss: -1543.4131

KeyboardInterrupt: 

### 6. Evaluation & Saving

In [ ]:
print("\n📊 Evaluating Final Performance...")
loss, accuracy = model.evaluate(test_generator)
print(f"✅ Final Test Accuracy: {accuracy * 100:.2f}%")
print(f"✅ Final Test Loss: {loss:.4f}")

# Save the model for future comparison
model.save('binary_brain_tumor_detector.h5')
print("\n💾 Model saved as 'binary_brain_tumor_detector.h5'")

### 7. Visualization

In [ ]:
# Plot Accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Binary Classification Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Binary Classification Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()